In [ ]:
# default_exp tree

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Tree module

> Provides functionality to construct and work with Tremaux tree out of the genome graph for tree sorting.

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
notebook2script()

Converted 00_init.ipynb.
Converted 01_graph.ipynb.
Converted 02_tree.ipynb.
Converted 03_synteny.ipynb.
Converted 04_utils.ipynb.
Converted 05_export.ipynb.
Converted index.ipynb.


In [ ]:
#exporti
import numpy as np

import pdb

import networkx as nx
from networkx.algorithms.operators.binary import compose
from networkx import NetworkXNoPath

from pangraph_constructor.utils import bidict

In [ ]:
#export
class AmbigouosTreeEdge(nx.AmbiguousSolution):
    """Raised if special edge in Tremaux tree cannot be identified as loop or bubble.
    """
    
class AmbigouosRootNode(nx.AmbiguousSolution):
    """Raised if more than 1 root node is found in a tree.
    """

In [ ]:
#export
class TremauxTree(nx.DiGraph):
    def __init__(self,graphToTree=None,parentGraph=None,byPath=True,**attr):
        if graphToTree is None:
            super().__init__(**attr)
        else:
            self.originalGraph = graphToTree
            self.parentGraph = parentGraph
            self.startingNodes = None
            
            treeCombined = self._generateTree(byPath)
            
            super().__init__(treeCombined,**attr)
            
            self.loopEdges = bidict()
            self.bubbleEdges = bidict()

            addEdges = self._getMissingEdges()

            self.processSpecialEdges(addEdges)

    def _getNodeStartFit(self):
        nodeStartTest = []
        for node in range(1,len(self.parentGraph.nodes)+1):
            nodeEdgePaths = [0]
            for edge,pathN in self.parentGraph.edgePaths.items():
                if edge[1]==node:
                    nodeEdgePaths.append(pathN)
            
            nodeStartTest.append(self.parentGraph.pathStarts[node-1]-max(nodeEdgePaths))
        return nodeStartTest
            
    def _generateTree(self,byPath=True):
        if byPath:
            inEdge = self.parentGraph.inPath
            outEdge = self.parentGraph.outPath
            nodeStartPath = self._getNodeStartFit()
            startingNodes = np.where(nodeStartPath==np.max(nodeStartPath))[0]+1
        else:
            inEdge = self.originalGraph.in_degree
            outEdge = self.originalGraph.out_degree
            startingNodes = np.arange(1,len(inEdge)+1)
            
        selectedOutPaths = np.array(outEdge)[startingNodes-1]    
        self.startingNodes = startingNodes[np.where(selectedOutPaths==np.max(selectedOutPaths))]
        selectedInPaths = np.array(inEdge)[startingNodes-1]
        self.startingNodes = startingNodes[np.where(selectedInPaths==np.min(selectedInPaths))].tolist()
#         self.startingNodes.sort(key=[self.parentGraph.pathStarts[node] for node in self.startingNodes])
        
        if len(self.startingNodes)>0:
            treeCombined = nx.DiGraph()
            for startNode in self.startingNodes:
                if startNode not in treeCombined:
                    treeCombined = compose(treeCombined,nx.dfs_tree(self.originalGraph,source=startNode))
            
            if byPath:
                self._preprocessPathTree(treeCombined)
            else:
                for toNode,numDeg in treeCombined.in_degree:
                    if numDeg>1:
                        for fromNode,_ in list(treeCombined.in_edges(nbunch=toNode))[1:]:
                            treeCombined.remove_edge(edge,toNode)
        else:
            raise ValueError("Something went wrong. I cannot find a single suitable starting node. It seems there is no nodes in the graph.")
        
        return treeCombined
    
    def _preprocessPathTree(self,treeCombined):
#         pdb.set_trace()
        for node in self.originalGraph.nodes:
            
            # get all edges coming to given node in the full graph
            graphInEdgeList = list(self.originalGraph.in_edges(node))
            # get all edges coming to a given node in the tremaux tree
            treeInEdgeList = list(treeCombined.in_edges(node))
#             print(f'Node: {node}')
#             print(f'Graph edges: {graphInEdgeList}')
#             print(f'Tree edges: {treeInEdgeList}')
            if len(graphInEdgeList)>0:# and len(treeInEdgeList)>0:
                # Remove self-cycle paths larger cycles will be removed later.
                graphInEdgeList = [edge for edge in graphInEdgeList if edge[0]!=edge[1]] # removed 'not nx.has_path(treeCombined,edge[1],edge[0]) and '
#                 print(graphInEdgeList)
                # Get number of paths passing each edge
                edgePaths = [self.parentGraph.edgePaths.get(edge,0) for edge in graphInEdgeList]
                # Find the edge with maximum number of paths passing
                edgeToKeep = graphInEdgeList[np.argmax(edgePaths)]
#                 print(f'Edge to keep: {edgeToKeep}')
                # If the best edge not in the tree already
                if edgeToKeep not in treeInEdgeList:
#                     print('Edge substituted!')
                    # Remove current incoming edge in the tree
                    treeCombined.remove_edges_from(treeInEdgeList)
                    # and replace it with newly identified best edge
                    treeCombined.add_edge(*edgeToKeep)
                    try:
                        # See if there was any cycle paths created
                        for path in nx.all_shortest_paths(treeCombined,edgeToKeep[1],edgeToKeep[0]):
#                             print(f'Path {path} severed!')
                            # For each cycle, severe the link outgoing from current node.
                            treeCombined.remove_edge(path[0],path[1])
                    except NetworkXNoPath:
                        # If no cycle path found, just carry on.
                        pass
#             print('#######')
                    
    def _getMissingEdges(self):
        graphEdges = set(list(self.originalGraph.edges))
        treeEdges = set(list(self.edges))
        return list((graphEdges - treeEdges))

    def processSpecialEdges(self,missingLinks):
        combinedGraph = nx.DiGraph(self)
        processedEdges = []
        for fromNode,toNode in missingLinks:
            edgeStatus = self.isBubbleLoop(fromNode,toNode,combinedGraph)
            if edgeStatus == -1:
#                 try:
#                     if self.parentGraph.edgePaths[(toNode,fromNode)]>self.parentGraph.edgePaths[(fromNode,toNode)]:
#                         self.loopEdges.add(fromNode,toNode)
#                     else:
#                         self.loopEdges.add(toNode,fromNode)
#                         if self.has_edge(toNode,fromNode):
#                             self.remove_edge(toNode,fromNode)
#                             self.add_edge(fromNode,toNode)
#                         else:
#                             self.bubbleEdges.add(fromNode,toNode)
#                             combinedGraph.add_edge(fromNode,toNode)
#                             processedEdges.append((toNode,fromNode))
#                 except KeyError:
#                     self.loopEdges.add(fromNode,toNode)
                self.loopEdges.add(fromNode,toNode)

            elif edgeStatus > 0 and (fromNode,toNode) not in processedEdges:
                self.bubbleEdges.add(fromNode,toNode)
                combinedGraph.add_edge(fromNode,toNode)

    # !!! Test this function with case on the paper!!! Incorrectly identify one of the links.
    def isBubbleLoop(self,start,end,treeBubbleGraph):
        '''
        return one of three statuses 
            - -1 (mark as loop link), ignored during sort, or
            - 1 (mark as bubble forward link), can changes sort order, or
            - 2 (mark as bubble cross link), can changes sort order, or 
            - raises error AmbiguousTreeEdge, which means that the special link class is not identified.
        '''
        
        # link forward jumping (bubble) at least one node (check length > 2), otherwise, raise error of repeat link.
        if start==end:
            return -1
        
        # Check which one of the two options to mark as loop.
        if start in dict(self[end]) or start in self.bubbleEdges.get(end,[]):
            return -1
        
        try:
            shortPath = nx.shortest_path(self,start,end)
        except nx.NetworkXNoPath:
            shortPath = None
        if shortPath is not None:
            if len(shortPath)>2:
                return 1
                
        else:
            # !!! test if there is a path from end to start (in combination of tremauxTree and bubbles)
            if (end in nx.ancestors(self,start)) or \
                (start in list(self[end].keys())) or \
                (start in self.bubbleEdges.get(end,[]) or \
                (nx.has_path(treeBubbleGraph,end,start))):
                return -1
            else:
                return 2
        
        raise AmbigouosTreeEdge('Path between start and end is only two nodes (not jumping nodes).')
        
    def draw(self):
        bubbleEdges = [(u,v) for u,ends in self.bubbleEdges.items() for v in ends ]
        loopEdges = [(u,v) for u,ends in self.loopEdges.items() for v in ends ]

        tempGraph = nx.DiGraph(self)
        tempGraph.add_edges_from(bubbleEdges+loopEdges)
        colours = []
        for edge in tempGraph.edges:
            if edge in self.edges:
                colours.append('blue')
            elif edge in bubbleEdges:
                colours.append('green')
            elif edge in loopEdges:
                colours.append('red')
        nx.draw(tempGraph,edge_color=colours,pos=nx.kamada_kawai_layout(tempGraph),with_labels=True)
    
    def draw_original(self):
        nx.draw(self.originalGraph,pos=nx.kamada_kawai_layout(self.originalGraph),with_labels=True)
    def getRootNodes(self):
        return self.startingNodes
#         inCond = dict([(node,in_degree==0) for node,in_degree in self.in_degree])

#         outCond = dict([(node,in_degree!=0) for node,in_degree in self.out_degree])
        
#         rootList = []
        
#         for node,deg_in in inCond.items():
#             if outCond[node] and deg_in:
#                 rootList.append(node)
        
#         if len(rootList)>1:
#             raise AmbigouosRootNode(f'Only 1 root node is expected, but {len(roorList)} are found')
            
#         return rootList[0]



In [ ]:
notebook2script()

Converted 00_init.ipynb.
Converted 01_graph.ipynb.
Converted 02_tree.ipynb.
Converted 03_synteny.ipynb.
Converted 04_utils.ipynb.
Converted index.ipynb.
